In [50]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
# Named Entity Recognition (NER) is a natural language processing task that involves 
# identifying and classifying named entities 
# (such as person names, locations, organizations, dates, etc.) in text.

In [51]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

In [52]:
# 학습용 말뭉치 데이터를 불러옴
corpus = read_file('train.txt')

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 3555
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302


In [53]:
corpus = read_file('train.txt')
print(corpus[0])
print(corpus[1])

[('1', '한편', 'NNG', 'O'), ('1', ',', 'SP', 'O'), ('2', 'AFC', 'SL', 'O'), ('2', '챔피언스', 'NNG', 'O'), ('2', '리그', 'NNG', 'O'), ('3', 'E', 'SL', 'B_OG'), ('3', '조', 'NNG', 'I'), ('3', '에', 'JKB', 'O'), ('4', '속하', 'VV', 'O'), ('4', 'ㄴ', 'ETM', 'O'), ('5', '포항', 'NNP', 'O'), ('6', '역시', 'MAJ', 'O'), ('7', '대회', 'NNG', 'O'), ('8', '8강', 'NNG', 'O'), ('9', '진출', 'NNG', 'O'), ('9', '이', 'JKS', 'O'), ('10', '불투명', 'NNG', 'O'), ('10', '하', 'VV', 'O'), ('10', '다', 'EC', 'O'), ('11', '.', 'SF', 'O')]
[('1', '2003', 'SN', 'B_DT'), ('1', '년', 'NNB', 'I'), ('2', '6', 'SN', 'I'), ('2', '월', 'NNB', 'I'), ('3', '14', 'SN', 'I'), ('3', '일', 'NNB', 'I'), ('4', '사직', 'NNG', 'O'), ('5', '두산', 'NNP', 'O'), ('5', '전', 'NNG', 'O'), ('6', '이후', 'NNG', 'O'), ('7', '박명환', 'NNP', 'B_PS'), ('7', '에게', 'JKB', 'O'), ('8', '당하', 'VV', 'O'), ('8', '았', 'EP', 'O'), ('8', '던', 'ETM', 'O'), ('9', '10', 'SN', 'O'), ('9', '연패', 'NNG', 'O'), ('10', '사슬', 'NNG', 'O'), ('10', '을', 'JKO', 'O'), ('11', '거의', 'MAG', 'O'), ('12'

In [54]:
print(sentences)

[['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.'], ['2003', '년', '6', '월', '14', '일', '사직', '두산', '전', '이후', '박명환', '에게', '당하', '았', '던', '10', '연패', '사슬', '을', '거의', '5', '년', '만', '에', '끊', '는', '의미', '있', '는', '승리', '이', '었', '다', '.'], ['AP', '통신', '은', '8', '일', '(', '이하', '한국', '시간', ')', '올라주원', ',', '유잉', '을', '비롯', '하', '아', '애드리언', '댄틀리', ',', '팻', '라일리', '감독', ',', '캐시', '러시', '감독', ',', 'TV', '해설가', '딕', '바이텔', ',', '디트로이트', '피스톤스', '의', '구단주', '윌리엄', '데이비드슨', '등', '이', '2008', '명예', '의', '전당', '헌액', '자', '로', '결정', '되', '었', '다고', '보', '아도', '하', '았', '다', '.'], ['(', '개막', '6', '연패', ')', '"', '짐', '릴랜드', '디트로이트', '감독', '은', '메이저리그', '에서', '손꼽히', '는', '명장', '이', '면서', '덕장', '이', '다', '.'], ['역시', '새로', '데려오', 'ㄴ', '돈트렐', '윌리스', '는', '6', '일', '경기', '에서', '삼진', '을', '하나', '도', '잡', '지', '않', '고', '볼넷', '7', '개', '를', '내주', '는', '엽기', '피칭', '을', '하', '었', '다', '.'], ['웽거', '감독', '은', '8', '일', '(', '이하', '한국

In [55]:
print(tags)

[['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B_DT', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'B_PS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_DT', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B_OG', 'I', 'O', 'B_DT', 'I', 'O', 'O', 'B_LC', 'O', 'O', 'B_PS', 'O', 'B_PS', 'O', 'O', 'O', 'O', 'B_PS', 'I', 'O', 'B_PS', 'I', 'O', 'O', 'B_PS', 'I', 'O', 'O', 'O', 'O', 'B_PS', 'I', 'O', 'B_OG', 'I', 'O', 'O', 'B_PS', 'I', 'O', 'O', 'B_DT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B_PS', 'I', 'B_OG', 'O', 'O', 'B_OG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'B_PS', 'I', 'O', 'B_DT', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B_PS', 'O', 'O', 'B_DT', 'I', 'O', 'O', 'B_LC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B_OG'

In [56]:
# 토크나이저 정의
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') # 첫 번째 인덱스에는 OOV 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower= False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)

In [57]:
# 단어 사전 및 태그 사전 크기
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 8
단어 사전 크기 : 13834


In [58]:
print(sent_tokenizer.word_index)

{'OOV': 1, '하': 2, '.': 3, '이': 4, '을': 5, '는': 6, 'ㄴ': 7, '다': 8, '의': 9, '에': 10, ',': 11, '를': 12, '은': 13, '았': 14, '고': 15, '(': 16, ')': 17, '었': 18, '가': 19, '아': 20, '에서': 21, '으로': 22, '있': 23, '일': 24, '되': 25, '로': 26, "'": 27, '어': 28, 'ㄹ': 29, '과': 30, '들': 31, '1': 32, '2': 33, '도': 34, '와': 35, '-': 36, '등': 37, '것': 38, '지': 39, '3': 40, 'ㄴ다': 41, '년': 42, '"': 43, '월': 44, '%': 45, '만': 46, '적': 47, '기': 48, '게': 49, '4': 50, '시': 51, '원': 52, '며': 53, '수': 54, '5': 55, '밝히': 56, '대하': 57, '주': 58, '말': 59, '다고': 60, '면': 61, '던': 62, '위하': 63, '전': 64, '오': 65, '까지': 66, '10': 67, '대': 68, '지나': 69, '부터': 70, '7': 71, ':': 72, '6': 73, '한국': 74, '받': 75, '않': 76, '명': 77, '경기': 78, '대표': 79, '보': 80, '지만': 81, '라고': 82, '시장': 83, '에게': 84, '따르': 85, '9': 86, '8': 87, '개': 88, '내': 89, '중': 90, '서울': 91, '‘': 92, '분': 93, '억': 94, '회': 95, '’': 96, '감독': 97, '의원': 98, '기록': 99, '제': 100, '~': 101, '11': 102, '습니다': 103, '팀': 104, '그': 105, 'ㄴ다고': 106, '이번': 107, '면서': 

In [59]:
print(len(sent_tokenizer.word_index))

13833


In [60]:
print(tag_tokenizer.word_index)

{'O': 1, 'I': 2, 'B_OG': 3, 'B_PS': 4, 'B_DT': 5, 'B_LC': 6, 'B_TI': 7}


In [61]:
sentences[0]

['한편',
 ',',
 'AFC',
 '챔피언스',
 '리그',
 'E',
 '조',
 '에',
 '속하',
 'ㄴ',
 '포항',
 '역시',
 '대회',
 '8강',
 '진출',
 '이',
 '불투명',
 '하',
 '다',
 '.']

In [62]:
tags[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'B_OG',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [63]:
# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0], len(x_train[0]))  # x_train[0] is the word_indexes of sentences[0] above.
print(y_train[0], len(y_train[0]))  # y_train[0] is the word_indexes of tags[0] above.

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3] 20
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 20


In [64]:
# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
index_to_ner[0] = 'PAD'

# Named Entity Recognition (NER) is a natural language processing task that involves 
# identifying and classifying named entities 
# (such as person names, locations, organizations, dates, etc.) in text.

In [65]:
print(index_to_word)  # only opposite order.   word_index <---> index_to_word

{1: 'OOV', 2: '하', 3: '.', 4: '이', 5: '을', 6: '는', 7: 'ㄴ', 8: '다', 9: '의', 10: '에', 11: ',', 12: '를', 13: '은', 14: '았', 15: '고', 16: '(', 17: ')', 18: '었', 19: '가', 20: '아', 21: '에서', 22: '으로', 23: '있', 24: '일', 25: '되', 26: '로', 27: "'", 28: '어', 29: 'ㄹ', 30: '과', 31: '들', 32: '1', 33: '2', 34: '도', 35: '와', 36: '-', 37: '등', 38: '것', 39: '지', 40: '3', 41: 'ㄴ다', 42: '년', 43: '"', 44: '월', 45: '%', 46: '만', 47: '적', 48: '기', 49: '게', 50: '4', 51: '시', 52: '원', 53: '며', 54: '수', 55: '5', 56: '밝히', 57: '대하', 58: '주', 59: '말', 60: '다고', 61: '면', 62: '던', 63: '위하', 64: '전', 65: '오', 66: '까지', 67: '10', 68: '대', 69: '지나', 70: '부터', 71: '7', 72: ':', 73: '6', 74: '한국', 75: '받', 76: '않', 77: '명', 78: '경기', 79: '대표', 80: '보', 81: '지만', 82: '라고', 83: '시장', 84: '에게', 85: '따르', 86: '9', 87: '8', 88: '개', 89: '내', 90: '중', 91: '서울', 92: '‘', 93: '분', 94: '억', 95: '회', 96: '’', 97: '감독', 98: '의원', 99: '기록', 100: '제', 101: '~', 102: '11', 103: '습니다', 104: '팀', 105: '그', 106: 'ㄴ다고', 107: '이번', 108: '

In [66]:
print(index_to_ner)

{1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI', 0: 'PAD'}


In [81]:
# 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)
print(x_train)
print('---------see elements of x_train---------')
print(x_train[0]) # padding at the end with 0
print(x_train[1])
print('-----------------------------------------')
print(y_train)

[[ 430 4417   16 ...    0    0    0]
 [5801    9  190 ...    0    0    0]
 [ 160  173    6 ...    0    0    0]
 ...
 [ 558  776   31 ...    0    0    0]
 [ 107 5813    6 ...  854   41    3]
 [ 403   42  253 ...   14    8    3]]
---------see elements of x_train---------
[ 430 4417   16  416   17    9 6582   13 6583    2   14    8    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[ 5801     9   190   526  2576     4    32    46  6192    52    21    32
    46  6288    52    11   357  2576     4    33    46 13393    52    21
    33    46 13394    52    22   388   839    25    18     8     3     0
     0     0     0     0]
-----------------------------------------
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  ...
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]]

 [[0 1 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  ...
  [1 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]
  [1 

In [68]:
# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)

# 출력 데이터를 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 8)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40, 8)


In [80]:
x_train[0]

array([ 430, 4417,   16,  416,   17,    9, 6582,   13, 6583,    2,   14,
          8,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0])

In [69]:
y_train[0] # one-hot encoded!

array([[0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 

In [9]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])

Epoch 1/10
23/23 [==============================] - 27s 564ms/step - loss: 0.7029 - accuracy: 0.8340
Epoch 2/10
23/23 [==============================] - 14s 597ms/step - loss: 0.3219 - accuracy: 0.8985
Epoch 3/10
23/23 [==============================] - 13s 559ms/step - loss: 0.2181 - accuracy: 0.9256
Epoch 4/10
23/23 [==============================] - 13s 571ms/step - loss: 0.1576 - accuracy: 0.9463
Epoch 5/10
23/23 [==============================] - 14s 600ms/step - loss: 0.1168 - accuracy: 0.9631
Epoch 6/10
23/23 [==============================] - 13s 567ms/step - loss: 0.0850 - accuracy: 0.9736
Epoch 7/10
23/23 [==============================] - 13s 562ms/step - loss: 0.0644 - accuracy: 0.9798
Epoch 8/10
23/23 [==============================] - 14s 625ms/step - loss: 0.0503 - accuracy: 0.9839
Epoch 9/10
23/23 [==============================] - 13s 586ms/step - loss: 0.0415 - accuracy: 0.9869
Epoch 10/10
23/23 [==============================] - 4s 72ms/step - loss: 0.2789 - accuracy

In [10]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
    return result

In [85]:
print(x_test[0])

[  69    7 1407   42 7891  581  206 2409   19 1560 4932    2   15  604
   66 2047   25   28 7892  344   13  271    5  358    4  121    8    3
    0    0    0    0    0    0    0    0    0    0    0    0]


In [11]:
# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test) # (711, 40) => model => (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER

# F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

23/23 [==============================] - 3s 59ms/step


c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.56      0.58      0.57       657
         _DT       0.87      0.91      0.89       335
         _LC       0.72      0.54      0.61       312
         _OG       0.68      0.57      0.62       481
         _PS       0.67      0.56      0.61       374
         _TI       0.87      0.73      0.79        66

   micro avg       0.68      0.62      0.65      2225
   macro avg       0.73      0.65      0.68      2225
weighted avg       0.68      0.62      0.65      2225

F1-score: 65.1%


In [88]:
print(pred_tags[0])
print(test_tags[0])

['B_DT', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['B_DT', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [95]:
# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])

print("새로운 유형의 시퀀스 : ", new_x)
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding="post", value=0, maxlen=max_len)
print("새로운 유형의 패딩된 시퀀스 : ", new_padded_seqs)
print(len(new_padded_seqs[0]))

새로운 유형의 시퀀스 :  [531, 307, 1476, 286, 1507, 6766, 1]
새로운 유형의 패딩된 시퀀스 :  [[ 531  307 1476  286 1507 6766    1    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
40


In [99]:
# NER 예측
p = model.predict(np.array([new_padded_seqs[0]]))
print('p : ', p)
p = np.argmax(p, axis=-1)
print('index of each maximum elements of p : ', p)
print('length of p :', len(p[0]))# 예측된 NER 인덱스값 추출
# only 7 is real values, rest is padding!
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)

1/1 [==============================] - 0s 90ms/step
p :  [[[2.73214180e-08 2.28569297e-05 1.57157810e-05 9.99806583e-01
   1.34252332e-04 8.51349071e-08 2.06082786e-05 1.12121468e-09]
  [4.35526104e-09 9.99004185e-01 9.93083930e-04 1.90553737e-06
   2.46271583e-07 6.33926106e-07 1.88421208e-08 2.04983088e-08]
  [3.10081116e-09 9.99984980e-01 1.27318481e-05 6.57313819e-07
   3.19438527e-07 1.15652642e-06 7.17836173e-08 1.97988133e-08]
  [3.79204238e-07 3.15336598e-04 1.68166832e-06 3.10450247e-07
   7.01600147e-05 9.99445736e-01 5.41633672e-05 1.12101821e-04]
  [7.23664186e-07 8.23735754e-05 9.31881135e-04 9.98041630e-01
   2.90985063e-05 8.76535125e-07 9.12381278e-04 8.97885684e-07]
  [9.85743536e-05 1.19162470e-01 8.75214756e-01 1.38519891e-03
   1.08524269e-04 5.72583755e-04 2.18238193e-03 1.27552589e-03]
  [1.52990583e-03 1.96334735e-01 7.69908130e-01 5.11308294e-03
   1.81432429e-03 4.03763447e-03 9.61556286e-03 1.16466377e-02]
  [1.19842604e-01 1.48641303e-01 1.23242982e-01 1.2100

In [104]:
for w, pred in zip(new_sentence, p[0]):
    print(w, pred)
    print("{:10} {:5}".format(w, index_to_ner[pred]))
    print('-' * 20)

삼성전자 3
삼성전자       B_OG 
--------------------
출시 1
출시         O    
--------------------
스마트폰 1
스마트폰       O    
--------------------
오늘 5
오늘         B_DT 
--------------------
애플 3
애플         B_OG 
--------------------
도전장 2
도전장        I    
--------------------
내밀다. 2
내밀다.       I    
--------------------
